In [ ]:
import pandas as pd

df = pd.read_csv('data.csv')  # read df from csv file

df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H;%M:%S')
df = df.drop(columns=['Date', 'Time'])
df.set_index('Datetime', inplace=True)

# using month and hour as features
df['Month'], df['Day'], df['Hour']  = df.index.month, df.index.day, df.index.hour 
print(df[:4])

In [ ]:
from System import Plant
from Policy import Policy
from LSTM import LSTM
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

dt = 1/60
plant = Plant(dt)
plant_comp = Plant(dt)
plant.reset(init_battery=7, init_peak=0, init_load=0)
plant_comp.reset(init_battery=7, init_peak=0, init_load=0)
policy = Policy()

control = []
battery = []
peak_load_comp = []
peak_load = []

for data in df[:10000].itertuples():
    # Index=Timestamp('2007-10-01 00:00:00'), Load=1.324, Month=10, Day=1, Hour=0
    u, l_hat = policy.select_action(plant)


    control.append(u)
    battery.append(plant_comp.battery.c)
    plant.step(u, data[1])

    plant_comp.step(-0.03, data[1])

    peak_load.append(plant.p)
    peak_load_comp.append(plant_comp.p)
    
print("Peak load:", plant.p, "Peak load without control:", plant_comp.p)

fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(battery)
ax2.plot(control)
fig.show()